<a href="https://colab.research.google.com/github/Nilanjan2223/EscapeScrape/blob/main/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jason

  Preparing metadata (setup.py) ... done
  Created wheel for jason: filename=jason-0.1.7-py3-none-any.whl size=2622 sha256=839b04a824490922abd7c7ba13ecb98533a89c1c17aaf19ecd4abbd7fe36781d
  Stored in directory: /root/.cache/pip/wheels/ac/9f/46/8de9cb09a2e1ce351d964e3f5df3e7d9cb7229522cb212ff1b
Successfully built jason


In [2]:
!pip install requests beautifulsoup4 pandas


In [3]:
!pip install playwright


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 10.8 MB/s eta 0:00:00


In [4]:
!playwright install

161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 56.1s161.3 MiB [] 0% 22.5s161.3 MiB [] 0% 29.2s161.3 MiB [] 0% 32.2s161.3 MiB [] 0% 30.5s161.3 MiB [] 0% 24.2s161.3 MiB [] 0% 21.0s161.3 MiB [] 1% 17.8s161.3 MiB [] 1% 15.8s161.3 MiB [] 1% 14.9s161.3 MiB [] 1% 13.9s161.3 MiB [] 2% 13.3s161.3 MiB [] 2% 12.7s161.3 MiB [] 2% 12.4s161.3 MiB [] 2% 12.7s161.3 MiB [] 2% 12.2s161.3 MiB [] 3% 11.9s161.3 MiB [] 3% 10.8s161.3 MiB [] 3% 11.1s161.3 MiB [] 4% 10.5s161.3 MiB [] 4% 10.3s161.3 MiB [] 4% 10.1s161.3 MiB [] 4% 10.0s161.3 MiB [] 5% 9.6s161.3 MiB [] 5% 9.2s161.3 MiB [] 5% 9.0s161.3 MiB [] 6% 8.9s161.3 MiB [] 6% 8.5s161.3 MiB [] 7% 8.2s161.3 MiB [] 7% 7.9s161.3 MiB [] 7% 7.6s161.3 MiB [] 8% 7.2s161.3 MiB [] 9% 6.8s161.3 MiB [] 9% 6.9s161.3 MiB [] 9% 6.6s161.3 MiB [] 10% 6.5s161.3 MiB [] 10% 6.2s161.3 MiB [] 11% 6.0s161.3 MiB [] 11% 5.9s161.3 MiB [] 12% 5.9s161.3 MiB [] 12% 5.7s161.3 MiB [] 13% 5.6s161.3 MiB [] 13% 5.5s161.3 MiB [] 14% 5.4s161.3 MiB [] 15% 5.4s161.3 MiB [] 15% 5.3s161.3 MiB [] 16% 5.2s161.3

In [5]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 44 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

In [6]:
!pip install nest_asyncio


In [7]:
from playwright.sync_api import sync_playwright
from dataclasses import dataclass, asdict, field
import pandas as pd
import os

@dataclass
class Business:
    name: str = None
    address: str = None
    website: str = None
    phone_number: str = None
    reviews_count: int = None
    reviews_average: float = None
    latitude: float = None
    longitude: float = None

@dataclass
class BusinessList:
    business_list: list[Business] = field(default_factory=list)

    def dataframe(self):
        return pd.json_normalize(
            (asdict(business) for business in self.business_list), sep="_"
        )

    def save_to_excel(self, filename):
        self.dataframe().to_excel(f"{filename}.xlsx", index=False)

    def save_to_csv(self, filename):
        self.dataframe().to_csv(f"{filename}.csv", index=False)

def extract_coordinates_from_url(url: str) -> tuple[float,float]:
    """helper function to extract coordinates from url"""
    coordinates = url.split('/@')[-1].split('/')[0]
    # return latitude, longitude
    return float(coordinates.split(',')[0]), float(coordinates.split(',')[1])


In [8]:
# Directly set your search parameters
search_list = ["Escape Room London"]  # Replace with your desired search term
total = 20  # Set the total number of results to scrape

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from playwright.async_api import async_playwright
async def main(search_list,total):
  async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.goto("https://www.google.com/maps")
        # wait is added for dev phase. can remove it in production
    await page.wait_for_timeout(5000)
    await page.wait_for_load_state('networkidle')

    for search_for_index, search_for in enumerate(search_list):
            print(f"-----\n{search_for_index} - {search_for}".strip())
            try:
              search_input= page.locator('input#searchboxinput')
              await search_input.fill(search_for)
              await page.wait_for_timeout(5000)
            except TimeoutError:
              print(f"TimeoutError: Could not find search input field for '{search_for}'. Skipping...")
              continue
            await page.keyboard.press("Enter")
            await page.wait_for_timeout(5000)

            # scrolling

            await page.hover('//a[contains(@href, "https://www.google.com/maps/place")]')

            # this variable is used to detect if the bot
            # scraped the same number of listings in the previous iteration

            previously_counted = set()
            while True:
                await page.mouse.wheel(0, 10000)
                await page.wait_for_timeout(3000)
                count = await page.locator('//a[contains(@href, "https://www.google.com/maps/place")]').all()
                total_count_set = set(count)

                new_unique_links = total_count_set - previously_counted
                previously_counted.update(new_unique_links)
                total_count = len(previously_counted)


                if total_count > total:
                    #listings = all_listings[:total]
                    all_listings = list(previously_counted)[:total]
                    print(f"Total Scraped: {len(all_listings)}")
                    break
                else:
                    print(f"Currently Scraped: {len(previously_counted)}")
                    await page.wait_for_timeout(3000)

            business_list = BusinessList()

            # scraping
            for listing in all_listings:
                try:
                    await listing.click()
                    await page.wait_for_timeout(5000)

                    name_xpath = '//h1[@class="DUwDvf lfPIob"]'
                    address_xpath = '//button[@data-item-id="address"]//div[contains(@class, "fontBodyMedium")]'
                    website_xpath = '//a[@data-item-id="authority"]//div[contains(@class, "fontBodyMedium")]'
                    phone_number_xpath = '//button[contains(@data-item-id, "phone:tel:")]//div[contains(@class, "fontBodyMedium")]'
                    review_count_xpath = '//span[contains(@aria-label, "reviews")]'
                    reviews_average_xpath = '//div[@jsaction="pane.reviewChart.moreReviews"]//div[@role="img"]'


                    business = Business()
                    names =  await page.query_selector(name_xpath)
                    address = await page.query_selector(address_xpath)
                    website = await page.query_selector(website_xpath)
                    phone_number = await page.query_selector(phone_number_xpath)
                    review_count = await page.query_selector(review_count_xpath)
                    reviews_average = await page.query_selector(reviews_average_xpath)


                    if names:
                        business.name = await names.inner_text()
                        print(f"Extracted name: {business.name}")
                    else:
                        print("No aria-label found for this listing.")
                        business.name = ""
                    if address:
                        business.address = await address.inner_text()
                    else:
                        business.address = ""
                    if website:
                        business.website = await website.inner_text()
                    else:
                        business.website = ""
                    if phone_number:
                        business.phone_number = await phone_number.inner_text()
                    else:
                        business.phone_number = ""
                    if review_count:
                        review_count_text = await review_count.inner_text()
                        review_count_text_clean = review_count_text.replace('(','').replace(')','').replace(',','').strip()
                        try:
                          business.reviews_count = int(review_count_text_clean)

                        except ValueError:
                          business.reviews_count = ""
                          print(f"Extracted review count: {business.reviews_count}")

                    else:
                        business.reviews_count = ""

                    if reviews_average:
                        reviews_average_text = await reviews_average.get_attribute('aria-label')
                        business.reviews_average =float(reviews_average_text
                            .split()[0]
                            .replace(',','.')
                            .strip())
                    else:
                        business.reviews_average = ""


                    business.latitude, business.longitude = extract_coordinates_from_url(page.url)

                    business_list.business_list.append(business)
                except Exception as e:
                    print(f'Error occured: {e}')

            #print(business_list.dataframe())
            print("Converting business list to DataFrame...")
            df = pd.json_normalize((asdict(business) for business in business_list.business_list), sep="_")
            print("DataFrame conversion successful, saving to Excel and CSV...")
            df.to_excel("google_maps_data.xlsx", index=False)
            df.to_csv("google_maps_data.csv", index=False)
            print("Business list saved successfully.")

search_list = ["Escape Rooms London "]  # Add your search queries
total = 50  # Set your desired total number of listings to scrape

asyncio.run(main(search_list,total))

-----
0 - Escape Rooms London
Currently Scraped: 12
Currently Scraped: 30
Currently Scraped: 50
Total Scraped: 50
Extracted name: Escape London
Extracted name: Escape Puzzles
Extracted name: Escape Rooms London - Escape Land
Extracted name: No Escape Oxford St
Extracted name: DNA VR
Extracted name: clueQuest - The Live Escape Game
Extracted name: No Escape (Enigma Escape)
Extracted name: Escape Room "The Butcher"
Extracted name: Escape Hunt Oxford Street
Extracted name: Escape from The Golden Hinde
Extracted name: Houdini's Escape Room Experience - Farringdon
Extracted name: Escape Room "The Butcher"
Extracted name: Looking for puzzle ideas for your next escape room game?
Extracted name: Game Over London
Extracted name: Omescape London
Extracted name: Game Over London
Extracted name: Escape Room "Cabin Fever"
Extracted name: Escape Room "Sweeney Todd"
Extracted name: Escape Room "Sweeney Todd"
Extracted name: Clue Adventures
Extracted name: ScreamWorks
Extracted name: SENSAS London
Ext